In [23]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, max, count, min, approx_count_distinct
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, StringType, IntegerType, DateType

spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("perform-aagregations")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # Spark 설정 옵션
         .getOrCreate()  # SparkSession 생성 또는 기존 세션 반환)

spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [24]:
df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기
      .option("header", "true")  # 첫 번째 행을 헤더로 사용
      .option("nullValue", "null")
      .option("dateFormat",  # 날짜 형식 지정 "LLLL d, y")
      .load(  # 파일 로드"../data/netflix_titles.csv"))

In [25]:
# 그룹화 the data by a column
grouped_df = df.groupBy(  # 그룹화"country")

In [26]:
# Count the number of rows in each group
count_df = grouped_df.count()  # 행 개수 계산
count_df.show()  # DataFrame 내용 출력

+--------------------+-----+
|             country|count|
+--------------------+-----+
|Peru, United Stat...|    1|
|India, United Kin...|    1|
|Japan, Canada, Un...|    1|
|United Kingdom, C...|    1|
|      India, Germany|    2|
|South Africa, Uni...|    1|
|              Russia|   15|
|United Kingdom, G...|    1|
|Chile, United Sta...|    1|
|South Africa, Angola|    1|
|United States, Po...|    1|
|  Philippines, Qatar|    1|
|Hong Kong, China,...|    1|
|  Germany, Sri Lanka|    1|
|Denmark, France, ...|    1|
|United States, Fr...|    1|
|United Kingdom, N...|    2|
|Australia, United...|    2|
|Brazil, France, G...|    1|
|United States, Ir...|    3|
+--------------------+-----+
only showing top 20 rows



In [27]:
# 적용 custom aggregation using max
max_release_df = grouped_df.agg(max(col(  # 컬럼 참조"date_added")))
max_release_df.show()  # DataFrame 내용 출력

+--------------------+--------------------+
|             country|     max(date_added)|
+--------------------+--------------------+
|                null|   September 9, 2021|
|     Ama K. Abebrese|  Kobina Amissah Sam|
|         Aziz Ansari|         Carla Gallo|
|            Chuck D.|     Desiree Densiti|
|       Dominic Costa|        Nick Ferraro|
|          Doug Plaut|    Cheyenne Jackson|
|     Francesc Orella|        Imanol Arias|
|  Henri-Noël Tabary"|              France|
|       James Toback"|       United States|
| Justin ""Alyssa ...|         Molly Ryman|
|  Lachion Buckingham|       Chasity Moore|
|  Leonardo Sbaraglia|     Francesc Orella|
|   Michael Cavalieri|     Walton Goggins"|
| Remilekun ""Remi...| Charles  ""Charl...|
|          Rob Morgan|     Shakira Barrera|
|       Sophia Loren"|       United States|
|     Tantoo Cardinal|   Robert Pastorelli|
|      Theo Campbell"|       United States|
| Tobechukwu ""iLL...|          Toni Tones|
| plus Whitney Cum...|          

In [28]:
release_date_gouped_df = (
    df.groupBy(  # 그룹화"country")
    .agg(  # 집계 함수 적용
        count("show_id").alias(  # 컬럼 별칭 설정"NumberOfReleases")
        ,max("date_added").alias(  # 컬럼 별칭 설정"LastReleaseDate")
        ,min("date_added").alias(  # 컬럼 별칭 설정"FirstReleaseDate")))

release_date_gouped_df.show(3)

+----------------+----------------+-------------------+-------------------+
|         country|NumberOfReleases|    LastReleaseDate|   FirstReleaseDate|
+----------------+----------------+-------------------+-------------------+
|            null|             830|  September 9, 2021|  December 14, 2018|
| Ama K. Abebrese|               1| Kobina Amissah Sam| Kobina Amissah Sam|
|     Aziz Ansari|               1|        Carla Gallo|        Carla Gallo|
+----------------+----------------+-------------------+-------------------+
only showing top 3 rows



### Pivot Tables

In [29]:
pivot_table = (df.groupBy(  # 그룹화"country").pivot("type").agg(count("show_id")))
pivot_table.show()  # DataFrame 내용 출력

+--------------------+-----+-------+
|             country|Movie|TV Show|
+--------------------+-----+-------+
|Peru, United Stat...|    1|   null|
|United Kingdom, C...| null|      1|
|India, United Kin...|    1|   null|
|      India, Germany|    2|   null|
|Japan, Canada, Un...| null|      1|
|South Africa, Uni...|    1|   null|
|              Russia|    1|     14|
|United Kingdom, G...| null|      1|
|Chile, United Sta...|    1|   null|
|  Philippines, Qatar|    1|   null|
|United States, Fr...| null|      1|
|Hong Kong, China,...|    1|   null|
|Denmark, France, ...|    1|   null|
|South Africa, Angola|    1|   null|
|United States, Po...| null|      1|
|  Germany, Sri Lanka|    1|   null|
|United Kingdom, N...|    2|   null|
|Australia, United...|    2|   null|
|United States, Ir...|    2|      1|
|Brazil, France, G...|    1|   null|
+--------------------+-----+-------+
only showing top 20 rows



### Approximate Aggregations

In [30]:
# Define a Schema
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드("Id", IntegerType(), True),
    StructField  # 구조체 필드("ProductId", StringType(), True),
    StructField  # 구조체 필드("UserId", StringType(), True),
    StructField  # 구조체 필드("ProfileName", StringType(), True),
    StructField  # 구조체 필드("HelpfulnessNumerator", StringType(), True),
    StructField  # 구조체 필드("HelpfulnessDenominator", StringType(), True),
    StructField  # 구조체 필드("Score", IntegerType(), True),
    StructField  # 구조체 필드("Time", StringType(), True),
    StructField  # 구조체 필드("Summary", StringType(), True),
    StructField  # 구조체 필드("Text", StringType(), True)])


review_df = (spark.read.format("csv")  # CSV 형식으로 데이터 읽기
      .option("header",True)
      .schema(schema)
      .load(  # 파일 로드"../data/Reviews.csv"))

In [31]:
# Approximate quantile calculation
quantiles = review_df.approxQuantile("Score", [0.25, 0.5, 0.75], 0.1)
print("Approximate Quantiles:", quantiles)

Approximate Quantiles: [4.0, 5.0, 5.0]


In [32]:
# Approximate distinct count calculation
approx_distinct_count = review_df.select(  # 컬럼 선택approx_count_distinct("ProductId", rsd=0.1).alias("approx_distinct_count"))
approx_distinct_count.show()  # DataFrame 내용 출력

+---------------------+
|approx_distinct_count|
+---------------------+
|                 3780|
+---------------------+



In [33]:
spark.stop()  # Spark 세션 종료 - 리소스 정리